In [447]:
import onnx
import numpy as np

onnx_model = onnx.load('weights/yolo-fastest.onnx')

In [448]:
onnx.__version__

'1.8.0'

In [449]:
for i in onnx_model.graph.node:
	print(i)    

 "module_list.0.BatchNorm2d.bias"
input: "module_list.0.BatchNorm2d.running_mean"
input: "module_list.0.BatchNorm2d.running_var"
output: "504"
name: "BatchNormalization_1"
op_type: "BatchNormalization"
attribute {
  name: "epsilon"
  f: 9.999999747378752e-05
  type: FLOAT
}
attribute {
  name: "momentum"
  f: 0.9700000286102295
  type: FLOAT
}

input: "504"
output: "505"
name: "LeakyRelu_2"
op_type: "LeakyRelu"
attribute {
  name: "alpha"
  f: 0.10000000149011612
  type: FLOAT
}

input: "505"
input: "module_list.1.Conv2d.weight"
output: "506"
name: "Conv_3"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "group"
  i: 1
  type: INT
}
attribute {
  name: "kernel_shape"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "pads"
  ints: 0
  ints: 0
  ints: 0
  ints: 0
  type: INTS
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}

input: "506"
input: "module_list.1.BatchNorm2d.weight"
input: "module_list.1.BatchNorm

In [ ]:

# graph = onnx_model.graph
# for tensor_info in [graph.node[0]]:
#     print(tensor_info.attribute)
#     att= tensor_info.attribute
#     for at in att:
#         print('get',getattr(at, "type", None))

#     if (getattr(at, "type", None)==2):
#         setattr(at, "type",2)
    
#     setattr(at, "type",7)
#     print('############',tensor_info.attribute)    
#     tp=getattr(graph.input[0], "type", None)
#     if getattr(tp, "tensor_type", None).elem_type==2:
#         getattr(tp, "tensor_type", None).elem_type=6


In [430]:
onnx_model = onnx.load('weights/yolo-fastest.onnx')

In [431]:
from onnx import numpy_helper

In [450]:
# Modify the initializer
changed_initializer = []
for i in range(len(onnx_model.graph.initializer)):
    ini = onnx_model.graph.initializer[i]
    if ini.data_type == 7:
        print('#'*50)
        print(ini)
        dims = ini.dims
        data_type = ini.data_type
        name = ini.name
        changed_initializer.append(name)
        raw_data = ini.raw_data
        array = numpy_helper.to_array(ini).astype(np.int32)
        print(array)
        numpy_byte = array.tobytes()
        print('numpy_byte:',numpy_byte)
        print(dims)
        print('-'*50)        

        new_ini = onnx.helper.make_tensor(
            name=name, 
            data_type=onnx.TensorProto.INT32, 
            dims=dims, 
            vals=numpy_byte, 
            raw=True)
        #new_i = make_tensor_f(name, onnx.TensorProto.INT32, dims, array)
        onnx_model.graph.initializer.remove(ini)
        onnx_model.graph.initializer.insert(i, new_ini)
        print(new_ini)

##################################################
dims: 1
data_type: 7
name: "1536"
raw_data: "\003\000\000\000\000\000\000\000"

[3]
numpy_byte: b'\x03\x00\x00\x00'
[1]
--------------------------------------------------
dims: 1
data_type: 6
name: "1536"
raw_data: "\003\000\000\000"

##################################################
dims: 1
data_type: 7
name: "1537"
raw_data: "\007\000\000\000\000\000\000\000"

[7]
numpy_byte: b'\x07\x00\x00\x00'
[1]
--------------------------------------------------
dims: 1
data_type: 6
name: "1537"
raw_data: "\007\000\000\000"

##################################################
dims: 1
data_type: 7
name: "1538"
raw_data: "\n\000\000\000\000\000\000\000"

[10]
numpy_byte: b'\n\x00\x00\x00'
[1]
--------------------------------------------------
dims: 1
data_type: 6
name: "1538"
raw_data: "\n\000\000\000"

##################################################
dims: 1
data_type: 7
name: "1539"
raw_data: "\020\000\000\000\000\000\000\000"

[16]
numpy_byte:

In [426]:
# Check the node influenced
for i in changed_initializer:
    print(i)
    for node in onnx_model.graph.node:
        if i in node.input:
            print('#'*50)
            print(node)

1536
##################################################
input: "737"
input: "1536"
input: "1537"
input: "1538"
input: "1539"
output: "742"
name: "Concat_231"
op_type: "Concat"
attribute {
  name: "axis"
  i: 0
  type: INT
}

1537
##################################################
input: "737"
input: "1536"
input: "1537"
input: "1538"
input: "1539"
output: "742"
name: "Concat_231"
op_type: "Concat"
attribute {
  name: "axis"
  i: 0
  type: INT
}

1538
##################################################
input: "737"
input: "1536"
input: "1537"
input: "1538"
input: "1539"
output: "742"
name: "Concat_231"
op_type: "Concat"
attribute {
  name: "axis"
  i: 0
  type: INT
}

1539
##################################################
input: "737"
input: "1536"
input: "1537"
input: "1538"
input: "1539"
output: "742"
name: "Concat_231"
op_type: "Concat"
attribute {
  name: "axis"
  i: 0
  type: INT
}

1540
##################################################
input: "1540"
output: "757"
name: "ConstantO

In [451]:
# Modify the nodes:
for n in range(len(onnx_model.graph.node)):
    node = onnx_model.graph.node[n]
    name = node.name
    inputs = node.input
    outputs = node.output
    op_type = node.op_type
    for att in node.attribute:
        if att.t.data_type == 7:
            print('#'*50)
            att_name = att.name
            print(node)
            print('-'*50)
            print(onnx.numpy_helper.to_array(att.t))
            numpy_array = onnx.numpy_helper.to_array(att.t).astype(np.int32)
            new_tensor =  numpy_helper.from_array(numpy_array)
            # X = onnx.helper.make_tensor_value_info(att_name, onnx.TensorProto.INT32, numpy_array)
            print('-'*50)
            print(new_tensor)
            new_node = onnx.helper.make_node(
                name=name,
                inputs=inputs,
                outputs=outputs,
                op_type=op_type,
                value=new_tensor
            )
            print(new_node)
            onnx_model.graph.node.remove(node)
            onnx_model.graph.node.insert(n, new_node)
    continue


##################################################
output: "731"
name: "Constant_228"
op_type: "Constant"
attribute {
  name: "value"
  t {
    data_type: 7
    raw_data: "\000\000\000\000\000\000\000\000"
  }
  type: TENSOR
}

--------------------------------------------------
0
--------------------------------------------------
data_type: 6
raw_data: "\000\000\000\000"

output: "731"
name: "Constant_228"
op_type: "Constant"
attribute {
  name: "value"
  t {
    data_type: 6
    raw_data: "\000\000\000\000"
  }
  type: TENSOR
}

##################################################
output: "745"
name: "Constant_234"
op_type: "Constant"
attribute {
  name: "value"
  t {
    dims: 1
    data_type: 7
    raw_data: "\004\000\000\000\000\000\000\000"
  }
  type: TENSOR
}

--------------------------------------------------
[4]
--------------------------------------------------
dims: 1
data_type: 6
raw_data: "\004\000\000\000"

output: "745"
name: "Constant_234"
op_type: "Constant"
attribute {


In [452]:
onnx.checker.check_model(onnx_model)

In [418]:
name = '1157'

print('#'*50,'\nFound in initializer: ')
for initializer in onnx_model.graph.initializer:
    if initializer.name == name:
        print(initializer)
        print(onnx.numpy_helper.to_array(initializer))

print('#'*50,'\nFound in output: ')
for node in onnx_model.graph.node:
    if name in node.output:
        print(node)

print('#'*50,'\nFound in input: ')
for node in onnx_model.graph.node:
    if name in node.input:
        print(node)
        # for att in node.attribute:
        #     print('-'*50)
        #     print(att)
        #     print(att.type)


################################################## 
Found in initializer: 
dims: 5
data_type: 6
name: "1157"
raw_data: "\001\000\000\000\003\000\000\000\007\000\000\000\024\000\000\000 \000\000\000"

[ 1  3  7 20 32]
################################################## 
Found in output: 
################################################## 
Found in input: 
input: "1144"
input: "1157"
output: "1158"
name: "Reshape_624"
op_type: "Reshape"



In [372]:
onnx_model.graph.input[0].type.tensor_type.elem_type

1

In [417]:
for n in range(len(onnx_model.graph.node)):
    node = onnx_model.graph.node[n]
    name = node.name
    if node.name != 'Reshape_624':
        continue
    inputs = node.input
    outputs = node.output
    op_type = node.op_type
    for att in node.attribute:
        if att.t.data_type == 7:
            print('#'*50)
            att_name = att.name
            print(node)
            print('-'*50)
            print(onnx.numpy_helper.to_array(att.t))
            numpy_array = onnx.numpy_helper.to_array(att.t).astype(np.int32)
            new_tensor =  numpy_helper.from_array(numpy_array)
            # X = onnx.helper.make_tensor_value_info(att_name, onnx.TensorProto.INT32, numpy_array)
            print('-'*50)
            print(new_tensor)
            new_node = onnx.helper.make_node(
                name=name,
                inputs=inputs,
                outputs=outputs,
                op_type=op_type,
                value=new_tensor
            )
            print(new_node)
            onnx_model.graph.node.remove(node)
            onnx_model.graph.node.insert(n, new_node)
    continue

In [453]:
onnx.save(onnx_model, 'weights/yolo-fastest-transfer.onnx')

In [457]:
batch_size = 1
# Input to the model
import torch
x = torch.randn(batch_size, 3, 320, 512, requires_grad=True)

import onnxruntime

ort_session = onnxruntime.InferenceSession("weights/yolo-fastest.onnx")
onnx_session = onnxruntime.InferenceSession("weights/yolo-fastest-opt.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
x_array = to_numpy(x)
ort_inputs = {ort_session.get_inputs()[0].name: x_array}
onnx_inputs = {onnx_session.get_inputs()[0].name: x_array}
ort_outs = ort_session.run(None, ort_inputs)
onnx_outs = onnx_session.run(None, onnx_inputs)


In [458]:
np.testing.assert_allclose(onnx_outs[0], ort_outs[0], rtol=1e-03, atol=1e-05)
np.testing.assert_allclose(onnx_outs[0], ort_outs[0], rtol=1e-03, atol=1e-05)


In [436]:
print(onnx.helper.printable_graph(onnx_model.graph))  # Print a human readable representation of the graph


graph torch-jit-export (
  %images[FLOAT, 1x3x320x512]
) initializers (
  %1536[INT64, 1]
  %1537[INT64, 1]
  %1538[INT64, 1]
  %1539[INT64, 1]
  %1540[INT64, 1]
  %1541[INT64, 1]
  %1542[INT64, 1]
  %1543[INT64, 1]
  %1544[INT64, 1]
  %1549[FLOAT, 4]
  %1550[INT64, 1]
  %1551[INT64, 1]
  %1552[INT64, 1]
  %1553[INT64, 1]
  %1554[INT64, 1]
  %1555[INT64, 1]
  %1556[INT64, 1]
  %1557[INT64, 1]
  %1558[INT64, 1]
  %module_list.0.BatchNorm2d.bias[FLOAT, 8]
  %module_list.0.BatchNorm2d.running_mean[FLOAT, 8]
  %module_list.0.BatchNorm2d.running_var[FLOAT, 8]
  %module_list.0.BatchNorm2d.weight[FLOAT, 8]
  %module_list.0.Conv2d.weight[FLOAT, 8x3x3x3]
  %module_list.1.BatchNorm2d.bias[FLOAT, 8]
  %module_list.1.BatchNorm2d.running_mean[FLOAT, 8]
  %module_list.1.BatchNorm2d.running_var[FLOAT, 8]
  %module_list.1.BatchNorm2d.weight[FLOAT, 8]
  %module_list.1.Conv2d.weight[FLOAT, 8x8x1x1]
  %module_list.10.BatchNorm2d.bias[FLOAT, 24]
  %module_list.10.BatchNorm2d.running_mean[FLOAT, 24]
  %mod

In [459]:
onnx_model = onnx.load('weights/yolo-fastest-opt.onnx')

In [455]:
from onnx_tf.backend import prepare

tf_rep = prepare(onnx_model)
tf_rep.export_graph("weights/tensorflow/model.pb")

INFO:tensorflow:Assets written to: weights/tensorflow/model.pb/assets


In [460]:
from onnx2keras import onnx_to_keras

# Call the converter (input - is the main model input name, can be different for your model)
k_model = onnx_to_keras(onnx_model, ['images'])

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> images.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> classes.
DEBUG:onnx2keras:Output 1 -> yolo1.
DEBUG:onnx2keras:Output 2 -> yolo2.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight 869 with shape (1,).
DEBUG:onnx2keras:Found weight 1157 with shape (5,).
DEBUG:onnx2keras:Found weight 870 with shape (1,).
DEBUG:onnx2keras:Found weight 874 with shape (1, 3, 1, 1, 2).
DEBUG:onnx2keras:Found weight 1118 with shape (3,).
DEBUG:onnx2keras:Found weight 742 with shape (5,).
DEBUG:onnx2keras:Found weight 868 with shape (1,).
DEBUG:onnx2keras:Found weight 1168 with shape (4,).
DEBUG:onnx2keras:Found weight 1408 with shape (1,).
DEBUG:onnx2keras:Found weight 1549 with shape (4,).
DEBUG:onnx2keras:Found weight 1128 with shape (0,).
DEBUG:onnx2keras:Found weight 1163 with shape (1,).
DEBUG:onnx2keras:F

KeyError: 'Resize'